In [ ]:
# Imprtation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

In [2]:
# Config / chemins
DATA_FILE = "qualite_air_togo.csv"
OUTPUT_DIR = Path("./output_qualite_air")
OUTPUT_DIR.mkdir(exist_ok=True)
PDF_PATH = OUTPUT_DIR / "Rapport_Qualite_Air_Togo.pdf"

In [3]:
# Figure paths
FIG_TS = OUTPUT_DIR / "evolution_pm25_par_region.png"
FIG_BAR = OUTPUT_DIR / "moyenne_pm25_par_region.png"
FIG_HEAT = OUTPUT_DIR / "heatmap_correlations.png"
FIG_SCATTER_PM25_TEMP = OUTPUT_DIR / "scatter_pm25_temperature.png"
FIG_SCATTER_PM25_HUM = OUTPUT_DIR / "scatter_pm25_humidite.png"

In [ ]:
# Seuil de référence pour PM2.5 (modifiable) utilisé pour compter les dépassements
PM25_SEUIL = 25.0  # µg/m³ 

In [5]:
# 1. Charger & nettoyer
df = pd.read_csv(DATA_FILE)

In [6]:
# Normaliser noms de colonnes (pratique)
df.columns = [c.strip() for c in df.columns]

In [7]:
# Vérifier types
# Convertir colonnes numériques en cas de soucis
num_cols = ["PM2.5 (µg/m³)", "CO2 (ppm)", "Ozone (µg/m³)", "Température (°C)", "Humidité (%)"]
for col in num_cols + ["Année"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [8]:
# Supprimer les lignes avec valeurs manquantes
missing_before = df.isnull().sum().sum()
df = df.dropna().reset_index(drop=True)
missing_after = df.isnull().sum().sum()

# 2. Indicateurs simples

In [9]:
# Moyennes par région
moyennes_region = df.groupby("Région")[num_cols].mean().round(2)

# Tendances temporelles (moyenne nationale par année)
tendance_annee = df.groupby("Année")[num_cols].mean().reset_index()

# Corrélations (toutes variables numériques)
corr = df[num_cols].corr().round(2)

# Comptage dépassements PM2.5 par région
df["depasse_pm25"] = df["PM2.5 (µg/m³)"] > PM25_SEUIL
depassements_par_region = df.groupby("Région")["depasse_pm25"].sum().sort_values(ascending=False)

# Meilleure / pire année pour PM2.5 (moyenne)
pm25_par_annee = df.groupby("Année")["PM2.5 (µg/m³)"].mean().sort_index()

# 3. Visualisations


In [10]:
sns.set(style="whitegrid")

In [11]:
# 3.1 Évolution du PM2.5 par région (time series)
plt.figure(figsize=(10,6))
for region, grp in df.groupby("Région"):
    grp_sorted = grp.sort_values("Année")
    plt.plot(grp_sorted["Année"], grp_sorted["PM2.5 (µg/m³)"], marker="o", label=region)
plt.xlabel("Année")
plt.ylabel("PM2.5 (µg/m³)")
plt.title("Évolution du PM2.5 par région (2020-2025)")
plt.legend()
plt.tight_layout()
plt.savefig(FIG_TS, dpi=150)
plt.close()

In [12]:
# 3.2 Bar chart: moyenne PM2.5 par région
plt.figure(figsize=(8,5))
order = moyennes_region["PM2.5 (µg/m³)"].sort_values(ascending=False).index
sns.barplot(x=moyennes_region.loc[order, "PM2.5 (µg/m³)"].values, y=order)
plt.xlabel("PM2.5 moyen (µg/m³)")
plt.title("PM2.5 moyen par région (2020-2025)")
plt.tight_layout()
plt.savefig(FIG_BAR, dpi=150)
plt.close()

In [13]:
# 3.3 Heatmap de corrélations
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, fmt=".2f", linewidths=0.5)
plt.title("Corrélations entre indicateurs")
plt.tight_layout()
plt.savefig(FIG_HEAT, dpi=150)
plt.close()

In [14]:
# 3.4 Scatter: PM2.5 vs Température
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="Température (°C)", y="PM2.5 (µg/m³)", hue="Région", s=80)
plt.title("PM2.5 vs Température")
plt.tight_layout()
plt.savefig(FIG_SCATTER_PM25_TEMP, dpi=150)
plt.close()

In [15]:
# 3.5 Scatter: PM2.5 vs Humidité
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x="Humidité (%)", y="PM2.5 (µg/m³)", hue="Région", s=80)
plt.title("PM2.5 vs Humidité")
plt.tight_layout()
plt.savefig(FIG_SCATTER_PM25_HUM, dpi=150)
plt.close()

# 4. Génération du rapport PDF (ReportLab)


In [ ]:
# Récuperer les styles de texte standard avec getSampleStyleSheet()
styles = getSampleStyleSheet()

# Créer un document PDF (SimpleDocTemplate) avec le chemin, le format 
# de page A4 et des marges personnalisées.
doc = SimpleDocTemplate(str(PDF_PATH), pagesize=A4, rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=18)

# rée une liste vide elements qui servira à stocker tous les composants 
# (titres, paragraphes, tableaux, images, etc.) qui seront ajoutés au PDF
elements = []


In [ ]:
# Titre
elements.append(Paragraph("Rapport — Qualité de l'air au Togo (2020-2025)", styles["Title"]))
elements.append(Spacer(1, 12))

In [ ]:
# Résumé des données
elements.append(Paragraph(f"Source : fichier fourni — analyses fictives pour démonstration.", styles["Normal"]))
elements.append(Spacer(1, 6))
elements.append(Paragraph(f"Lignes initiales du fichier : {len(pd.read_csv(DATA_FILE))} — lignes utilisées après nettoyage : {len(df)}", styles["Normal"]))
elements.append(Spacer(1, 12))

In [ ]:
# Table: Moyennes par région (PM2.5, CO2, Ozone, Temp, Humidité)
elements.append(Paragraph("Moyennes par région (2020-2025)", styles["Heading2"]))
table_data = [ ["Région"] + list(moyennes_region.columns) ]
for region, row in moyennes_region.iterrows():
    table_data.append([region] + [row[c] for c in moyennes_region.columns])

t = Table(table_data, hAlign="LEFT", repeatRows=1)
t.setStyle(TableStyle([
    ("BACKGROUND",(0,0),(-1,0),colors.HexColor("#2E8B57")),
    ("TEXTCOLOR",(0,0),(-1,0),colors.white),
    ("ALIGN",(0,0),(-1,-1),"CENTER"),
    ("GRID",(0,0),(-1,-1),0.5,colors.grey),
    ("FONTSIZE",(0,0),(-1,-1),8),
]))
elements.append(t)
elements.append(Spacer(1,12))

In [ ]:
# Dépassements PM2.5
elements.append(Paragraph(f"Dépassements PM2.5 > {PM25_SEUIL} µg/m³ par région (nombre d'années)", styles["Heading2"]))
dep_table = [["Région", f"Nb. dépassements (> {PM25_SEUIL})"]]
for region, val in depassements_par_region.items():
    dep_table.append([region, int(val)])
t2 = Table(dep_table, hAlign="LEFT")
t2.setStyle(TableStyle([("GRID",(0,0),(-1,-1),0.5,colors.grey),("ALIGN",(0,0),(-1,-1),"CENTER")]))
elements.append(t2)
elements.append(Spacer(1,12))

In [ ]:
# Corrélations (petit tableau)
elements.append(Paragraph("Corrélations (extrait)", styles["Heading2"]))
corr_table = [["", *corr.columns]]
for idx in corr.index:
    corr_table.append([idx] + list(corr.loc[idx].values))
t3 = Table(corr_table, hAlign="LEFT")
t3.setStyle(TableStyle([("GRID",(0,0),(-1,-1),0.5,colors.grey),("FONTSIZE",(0,0),(-1,-1),8)]))
elements.append(t3)
elements.append(Spacer(1,12))

In [ ]:
# Insérer figures (images)
elements.append(Paragraph("Figures (voir ci-dessous)", styles["Heading2"]))
elements.append(Spacer(1,6))

In [ ]:

#  Aide pour insérer une image avec la largeur max
def add_image(path, width=450):
    try:
        img = Image(str(path))
        # Correction du redimensionnement : garder le ratio d'origine
        ratio = width / img.imageWidth
        img.drawWidth = width
        img.drawHeight = img.imageHeight * ratio
        elements.append(img)
        elements.append(Spacer(1,12))
    except Exception as e:
        elements.append(Paragraph(f"Impossible d'ajouter l'image {path.name}: {e}", styles["Normal"]))

In [ ]:
# Ajout des images (en ordre d'apparition)
for fig in [FIG_TS, FIG_BAR, FIG_HEAT, FIG_SCATTER_PM25_TEMP, FIG_SCATTER_PM25_HUM]:
    if fig.exists():
        elements.append(Paragraph(fig.name, styles["Normal"]))
        elements.append(Spacer(1,6))
        try:
            img = Image(str(fig))
            # Correction du redimensionnement : garder le ratio d'origine
            ratio = 450 / img.imageWidth
            img.drawWidth = 450
            img.drawHeight = img.imageHeight * ratio
            elements.append(img)
            elements.append(Spacer(1,12))
        except Exception as e:
            elements.append(Paragraph(f"Erreur insertion image {fig.name}: {e}", styles["Normal"]))
            elements.append(Spacer(1,6))

In [ ]:
# Conclusion / Insights (générés automatiquement)
elements.append(Paragraph("Conclusions & recommandations (automatiques)", styles["Heading2"]))
insights = []

# Région la plus polluée (moyenne PM2.5)
top_region = moyennes_region["PM2.5 (µg/m³)"].idxmax()
top_val = moyennes_region["PM2.5 (µg/m³)"].max()
insights.append(f"- Région avec PM2.5 moyen le plus élevé : {top_region} ({top_val} µg/m³).")

# Corrélations clés (extraction simple)
corr_pm25_temp = corr.loc["PM2.5 (µg/m³)", "Température (°C)"] if "Température (°C)" in corr.columns else np.nan
corr_pm25_hum = corr.loc["PM2.5 (µg/m³)", "Humidité (%)"] if "Humidité (%)" in corr.columns else np.nan
insights.append(f"- Corrélation PM2.5 vs Température : {corr_pm25_temp}")
insights.append(f"- Corrélation PM2.5 vs Humidité : {corr_pm25_hum}")

insights.append(f"- Nombre d'occurrences où PM2.5 > {PM25_SEUIL} µg/m³ par région (voir tableau ci-dessus).")
insights.append("- Recommandations : installer des capteurs supplémentaires dans les régions les plus touchées, lancer des campagnes de sensibilisation, prioriser la réduction des sources d'émission locales (trafic, déchets brûlés, industries).")

for para in insights:
    elements.append(Paragraph(para, styles["Normal"]))
    elements.append(Spacer(1,6))

In [ ]:

# Générer le PDF
doc.build(elements)

# 5. Résumé console


In [18]:
print("== Analyse terminée ==")
print(f"Fichier de données : {DATA_FILE}")
print(f"Lignes avant nettoyage : {len(pd.read_csv(DATA_FILE))}, après nettoyage : {len(df)}")
print(f"PDF généré : {PDF_PATH}")
print("Figures créées :")
for f in [FIG_TS, FIG_BAR, FIG_HEAT, FIG_SCATTER_PM25_TEMP, FIG_SCATTER_PM25_HUM]:
    print(" -", f)

== Analyse terminée ==
Fichier de données : qualite_air_togo.csv
Lignes avant nettoyage : 30, après nettoyage : 30
PDF généré : output_qualite_air\Rapport_Qualite_Air_Togo.pdf
Figures créées :
 - output_qualite_air\evolution_pm25_par_region.png
 - output_qualite_air\moyenne_pm25_par_region.png
 - output_qualite_air\heatmap_correlations.png
 - output_qualite_air\scatter_pm25_temperature.png
 - output_qualite_air\scatter_pm25_humidite.png
